# Bygg en klassificeringsmodell: Utsökta asiatiska och indiska rätter


## Klassificerare för kök 2

I denna andra lektion om klassificering kommer vi att utforska `fler sätt` att klassificera kategoriska data. Vi kommer också att lära oss om konsekvenserna av att välja en klassificerare framför en annan.

### [**Quiz före föreläsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Förkunskaper**

Vi antar att du har slutfört de tidigare lektionerna eftersom vi kommer att bygga vidare på några koncept vi lärde oss tidigare.

För denna lektion behöver vi följande paket:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) är en [samling av R-paket](https://www.tidyverse.org/packages) som är utformade för att göra datavetenskap snabbare, enklare och roligare!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) är ett [ramverk av paket](https://www.tidymodels.org/packages/) för modellering och maskininlärning.

-   `themis`: [themis-paketet](https://themis.tidymodels.org/) tillhandahåller extra receptsteg för att hantera obalanserad data.

Du kan installera dem med:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativt kan skriptet nedan kontrollera om du har de paket som krävs för att slutföra denna modul och installera dem åt dig om de saknas.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. En klassificeringskarta**

I vår [föregående lektion](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) försökte vi besvara frågan: hur väljer vi mellan flera modeller? Till stor del beror det på egenskaperna hos datan och typen av problem vi vill lösa (till exempel klassificering eller regression).

Tidigare lärde vi oss om de olika alternativen du har när du klassificerar data med hjälp av Microsofts fusklapp. Python's Machine Learning-ramverk, Scikit-learn, erbjuder en liknande men mer detaljerad fusklapp som kan hjälpa dig att ytterligare begränsa dina estimatorer (ett annat ord för klassificerare):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tips: [besök den här kartan online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) och klicka längs vägen för att läsa dokumentationen.
>
> [Tidymodels referenssida](https://www.tidymodels.org/find/parsnip/#models) erbjuder också utmärkt dokumentation om olika typer av modeller.

### **Planen** 🗺️

Den här kartan är väldigt användbar när du har en tydlig förståelse för din data, eftersom du kan "vandra" längs dess vägar mot ett beslut:

-   Vi har \>50 prover

-   Vi vill förutsäga en kategori

-   Vi har märkt data

-   Vi har färre än 100K prover

-   ✨ Vi kan välja en Linear SVC

-   Om det inte fungerar, eftersom vi har numerisk data

    -   Kan vi prova en ✨ KNeighbors Classifier

        -   Om det inte fungerar, prova ✨ SVC och ✨ Ensemble Classifiers

Det här är en väldigt användbar väg att följa. Nu ska vi dyka rakt in i det med [tidymodels](https://www.tidymodels.org/) modelleringsramverket: en konsekvent och flexibel samling av R-paket utvecklade för att främja god statistisk praxis 😊.

## 2. Dela upp data och hantera obalanserade dataset.

Från våra tidigare lektioner lärde vi oss att det fanns en uppsättning vanliga ingredienser över våra kök. Dessutom fanns det en ganska ojämn fördelning i antalet kök.

Vi kommer att hantera detta genom att

-   Ta bort de vanligaste ingredienserna som skapar förvirring mellan olika kök, med hjälp av `dplyr::select()`.

-   Använda ett `recipe` som förbehandlar data för att göra den redo för modellering genom att tillämpa en `over-sampling`-algoritm.

Vi tittade redan på detta i den tidigare lektionen, så det här borde gå som en dans 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Hantera obalanserad data

Obalanserad data påverkar ofta modellens prestanda negativt. Många modeller presterar bäst när antalet observationer är lika, och har därför en tendens att ha svårt med obalanserad data.

Det finns huvudsakligen två sätt att hantera obalanserade datasätt:

-   lägga till observationer till minoritetsklassen: `Över-sampling`, t.ex. med hjälp av en SMOTE-algoritm som syntetiskt genererar nya exempel av minoritetsklassen genom att använda närmaste grannar till dessa fall.

-   ta bort observationer från majoritetsklassen: `Under-sampling`

I vår tidigare lektion visade vi hur man hanterar obalanserade datasätt med hjälp av ett `recept`. Ett recept kan ses som en ritning som beskriver vilka steg som ska tillämpas på ett datasätt för att göra det redo för dataanalys. I vårt fall vill vi ha en jämn fördelning av antalet kök i vår `träningsuppsättning`. Låt oss sätta igång!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nu är vi redo att träna modeller 👩‍💻👨‍💻!

## 3. Utöver multinomiala regressionsmodeller

I vår tidigare lektion tittade vi på multinomiala regressionsmodeller. Låt oss utforska några mer flexibla modeller för klassificering.

### Support Vector Machines

I klassificeringssammanhang är `Support Vector Machines` en maskininlärningsteknik som försöker hitta ett *hyperplan* som "bäst" separerar klasserna. Låt oss titta på ett enkelt exempel:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ separerar inte klasserna. H2~ gör det, men endast med en liten marginal. H3~ separerar dem med maximal marginal.

#### Linjär Support Vector Classifier

Support-Vector clustering (SVC) är en del av Support-Vector-maskinerna inom ML-tekniker. I SVC väljs hyperplanet för att korrekt separera `de flesta` av träningsobservationerna, men `kan felklassificera` några observationer. Genom att tillåta vissa punkter att vara på fel sida blir SVM mer robust mot avvikelser och därmed bättre på att generalisera till ny data. Parametern som reglerar denna överträdelse kallas `cost` och har ett standardvärde på 1 (se `help("svm_poly")`).

Låt oss skapa en linjär SVC genom att sätta `degree = 1` i en polynomisk SVM-modell.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nu när vi har sammanställt förbehandlingsstegen och modellspecifikationen i ett *arbetsflöde*, kan vi gå vidare och träna den linjära SVC och samtidigt utvärdera resultaten. För prestandamått, låt oss skapa en uppsättning mått som kommer att utvärdera: `accuracy`, `sensitivity`, `Positive Predicted Value` och `F Measure`.

> `augment()` kommer att lägga till kolumn(er) för förutsägelser till den angivna datan.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support Vector Machine (SVM) är en vidareutveckling av support vector classifier för att hantera en icke-linjär gräns mellan klasserna. I grund och botten använder SVMs *kernel-tricket* för att utöka funktionsutrymmet och anpassa sig till icke-linjära relationer mellan klasser. En populär och mycket flexibel kernel-funktion som används av SVMs är *Radial basis function.* Låt oss se hur den presterar på vår data.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mycket bättre 🤩!

> ✅ Vänligen se:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> för vidare läsning.

### Närmaste granne-klassificerare

*K*-närmsta granne (KNN) är en algoritm där varje observation förutsägs baserat på dess *likhet* med andra observationer.

Låt oss anpassa en till vår data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Det verkar som att den här modellen inte presterar särskilt bra. Förmodligen kan modellens prestanda förbättras genom att ändra argumenten (se `help("nearest_neighbor")`). Se till att testa detta.

> ✅ Vänligen se:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> för att lära dig mer om *K*-Närmaste Grannar-klassificerare.

### Ensembleklassificerare

Ensemblealgoritmer fungerar genom att kombinera flera basmodeller för att skapa en optimal modell antingen genom:

`bagging`: att använda en *medelvärdesfunktion* på en samling av basmodeller

`boosting`: att bygga en sekvens av modeller som bygger på varandra för att förbättra den prediktiva prestandan.

Låt oss börja med att testa en Random Forest-modell, som bygger en stor samling beslutsträd och sedan använder en medelvärdesfunktion för att skapa en bättre övergripande modell.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bra jobbat 👏!

Låt oss också experimentera med en Boosted Tree-modell.

Boosted Tree definierar en ensemblemetod som skapar en serie sekventiella beslutsträd där varje träd beror på resultaten från tidigare träd i ett försök att gradvis minska felet. Den fokuserar på vikterna för felklassificerade objekt och justerar passformen för nästa klassificerare för att korrigera.

Det finns olika sätt att passa denna modell (se `help("boost_tree")`). I detta exempel kommer vi att passa Boosted trees via `xgboost`-motorn.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Vänligen se:
>
> -   [Machine Learning för samhällsvetare](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning med R](https://bradleyboehmke.github.io/HOML/)
>
> -   [En introduktion till statistisk inlärning med applikationer i R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Utforskar AdaBoost-modellen som är ett bra alternativ till xgboost.
>
> för att lära dig mer om ensembleklassificerare.

## 4. Extra - jämföra flera modeller

Vi har anpassat ganska många modeller i denna labb 🙌. Det kan bli tröttsamt eller jobbigt att skapa många arbetsflöden från olika uppsättningar av förbehandlingsmetoder och/eller modellspecifikationer och sedan beräkna prestandamåtten en efter en.

Låt oss se om vi kan lösa detta genom att skapa en funktion som anpassar en lista med arbetsflöden på träningsuppsättningen och sedan returnerar prestandamåtten baserat på testuppsättningen. Vi kommer att använda `map()` och `map_dfr()` från paketet [purrr](https://purrr.tidyverse.org/) för att tillämpa funktioner på varje element i en lista.

> [`map()`](https://purrr.tidyverse.org/reference/map.html)-funktioner låter dig ersätta många for-loopar med kod som både är mer kortfattad och lättare att läsa. Det bästa stället att lära sig om [`map()`](https://purrr.tidyverse.org/reference/map.html)-funktionerna är [kapitlet om iteration](http://r4ds.had.co.nz/iteration.html) i R för data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) paketet gör det möjligt för användare att skapa och enkelt anpassa ett stort antal modeller, men är främst utformat för att fungera med omprovningstekniker som `cross-validation`, en metod vi ännu inte har täckt.

## **🚀Utmaning**

Var och en av dessa tekniker har ett stort antal parametrar som du kan justera, till exempel `cost` i SVMs, `neighbors` i KNN, `mtry` (Slumpmässigt Valda Prediktorer) i Random Forest.

Undersök standardparametrarna för var och en och fundera på vad justering av dessa parametrar skulle innebära för modellens kvalitet.

För att ta reda på mer om en specifik modell och dess parametrar, använd: `help("model")` t.ex. `help("rand_forest")`

> I praktiken brukar vi *estimera* de *bästa värdena* för dessa genom att träna många modeller på en `simulerad datamängd` och mäta hur bra alla dessa modeller presterar. Denna process kallas **tuning**.

### [**Quiz efter föreläsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Granskning & Självstudier**

Det finns mycket facktermer i dessa lektioner, så ta en stund att gå igenom [denna lista](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) med användbar terminologi!

#### TACK TILL:

[`Allison Horst`](https://twitter.com/allison_horst/) för att ha skapat de fantastiska illustrationerna som gör R mer välkomnande och engagerande. Hitta fler illustrationer i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) och [Jen Looper](https://www.twitter.com/jenlooper) för att ha skapat den ursprungliga Python-versionen av denna modul ♥️

Lycka till med lärandet,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Konstverk av @allison_horst</figcaption>



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
